In [3]:
'''
To apply the previous simulations to multiple instrument portfolios, all that has to be added is accounting
for correlation between risk factors and positions of each security

To accomplish this, model a derivative position and a portfolio of derivatives
'''


'\nTo apply the previous simulations to multiple instrument portfolios, all that has to be added is accounting\nfor correlation between risk factors and positions of each security\n\nTo accomplish this, model a derivative position and a portfolio of derivatives\n'

In [4]:
# Imports from previous chapters
class market_environment(object):
    '''
    class to model a market environment relevant for valuation
    
    Attributes
    ===========
    name: string - name of the market environment
    pricing_date: datetime - date of the market environment
    
    Methods
    =========
    add_constant: adds a constant market parameter
    get_constant: get constant
    add_lists: adds a list (of underlyings)
    get_list: gets a list
    add_curve: adds a market curve (i.e. yield curve)
    get_curve: gets a market curve
    add_environment: upserts whole market environments with constants, lists, curves
    
    '''
    def __init__(self, name, pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self, key, constant):
        self.constants[key] = constant
    
    def get_constant(self, key):
        return self.constants[key]
    
    def add_list(self, key, list_object):
        self.lists[key] = list_object
        
    def get_list(self, key):
        return self.lists[key]
    
    def add_curve(self, key, curve):
        self.curves[key] = curve
        
    def get_curve(self, key):
        return self.curves[key]
    
    def add_environment(self, env):
        # overwrite values for class if they exist
        self.constants.update(env.constants)
        self.lists.update(env.lists)
        self.curves.update(env.curves)
        

In [5]:
'''
This is useful for calculating the general discount factor, in this case
exp(-r * t) at time t
Modeling this as a class:
'''
class constant_short_rate(object):
    '''
    Class for constant short rate discounting.
    
    Attributes
    ===========
    name: string - name of the object
    short_rate: float (positive) - constant rate for discounting
    
    Results
    ==========
    get_discount_factors: get discount factors given a list/array of timestamp objects
    '''
    
    def __init__(self, name, short_rate):
        self.name = name
        self.short_rate = short_rate
        if(short_rate < 0):
            raise ValueError('Short rate negative.')
    
    def get_discount_factors(self, date_list, dtobjects=True):
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * np.sort(-dlist)) # take negative of time fractions
        return np.array((date_list, dflist)).T

In [6]:
'''
Now build from a generic simulation class towards specific applications

'''
import numpy as np
import pandas as pd
class simulation_class(object):
    '''
    Provide base methods for simulation classes
    
    Params
    =========
    name: str - name of the object
    mar_env: market_environment - a market environment
    corr: bool - is this object correlated with another modeled object?
    
    Methods
    =========
    generated_time_grid: returns discretized time intervals for simulation
    get_instrument_values: returns current values of security simulated (as ndarray)
    '''
    
    def __init__(self, name, mar_env, corr):
        self.name = name
        self.pricing_date = mar_env.pricing_date
        self.initial_value = mar_env.get_constant('initial_value')
        self.volatility = mar_env.get_constant('volatility')
        self.final_date = mar_env.get_constant('final_date')
        self.currency = mar_env.get_constant('currency')
        self.frequency = mar_env.get_constant('frequency')
        self.paths = mar_env.get_constant('paths')
        self.discount_curve = mar_env.get_curve('discount_curve')
        try:
            # If there is a time grid in mar_env take it
            self.time_grid = mar_env.get_list('time_grid')
        except:
            self.time_grid = None
        try:
            # if there are special dates, add those
            self.special_dates = mar_env.get_list('special_dates')
        except:
            self.special_dates = []
        self.instrument_values = None
        self.correlated = corr
        if corr is True:
            # only needed when risk factors are correlated
            self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
            self.rn_set = mar_env.get_list('rn_set')[self.name]
            self.random_numbers = mar_env.get_list('random_numbers')
        
    def generate_time_grid(self):
        start = self.pricing_date
        end = self.final_date
        # use pandas date range func to generate time grid
        # freq is pandas frequency char
        time_grid = pd.date_range(start=start, end=end, freq=self.frequency).to_pydatetime()
        time_grid = list(time_grid)
        # add begin, end, special
        if start not in time_grid:
            time_grid.insert(0, start)
            
        if end not in time_grid:
            time_grid.append(end)
        if len(self.special_dates) > 0:
            # add all
            time_grid.extend(self.special_dates)
            # de-dup
            time_grid = list(set(time_grid))
            # sort
            time_grid.sort()
        # cast back to np array
        self.time_grid = np.array(time_grid)
    
    def get_instrument_values(self, fixed_seed=True):
        if self.instrument_values is None:
            # only initiative simulation if no values
            self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
        elif fixed_seed is False:
            # also resim if not a fixed seed (i.e. monte carlo)
            self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
        # Otherwise all set so return what was simulated already
        return self.instrument_values

In [7]:
'''
Set up the Euler Discretizatin of the Geometric Brownian Motion PDE
'''
class geometric_brownian_motion(simulation_class):
    '''
    Class to generate simulated paths based on the Black-Scholes-Merton GBM model
    
    Attributes
    =========
    name: string - name of obj
    mar_env: market_environment instance
    corr: Boolean - is this correlated to other modeled objects?
    
    Methods
    =========
    update: updates parameters
    generate_paths: returns Monte Carlo paths given env
    '''
    def __init__(self, name, mar_env, corr=False):
        super(geometric_brownian_motion, self).__init__(name, mar_env, corr)
        
    def update(self, initial_value=None, volatility=None, final_date=None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None
        
    def generate_paths(self, fixed_seed=False, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        # Grid size
        M = len(self.time_grid)
        # Number of Paths for Monte
        I = self.paths
        # ndArray of the Monte Carlo shape (time steps by number of evolutions)
        paths = np.zeros((M, I))
        # initialize first step of each trial with initial value
        paths[0] = self.initial_value

        if not self.correlated:
            # if not correlated, go random
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            # if correlated, use correlation random numbers object from market_env
            rand = self.random_numbers

        short_rate = self.discount_curve.short_rate
        # short rate for drift process
        
        for t in range(1, len(self.time_grid)):
            # select right idx from random number set
            if not self.correlated:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
        
            # difference between two dates as a year fraction (infinitisemal for Euler discretization)
            dt = (self.time_grid[t]- self.time_grid[t-1]).days / day_count        

            paths[t] = paths[t-1] * np.exp((short_rate - 0.5 * self.volatility ** 2) * dt +
                                          self.volatility * np.sqrt(dt) * ran)
            # ran is Brownian motion evolution
            # dt is time slice (weighting for this slice's evolution)
        
        self.instrument_values = paths
            

In [2]:
class derivatives_position(object):
    '''
    Class to model a derivatives position
    
    Attributes
    =========
    name: str - name of object
    quantity: float - number of assets making up position
    underlying: string - security derivative is on
    mar_env: market environment instance
    otype: str - valuation class to use
    payoff_func: str - Python payoff func to use (uses eval())
    
    Methods
    =========
    get_info: Prints information about this position
    '''
    def __init__(self, name, quantity, underlying, mar_env, otype, payoff_func):
        self.name = name
        self.quantity = quantity
        self.underlying = underlying
        self.mar_env = mar_env
        self.otype = otype
        self.payoff_func = payoff_func
        
    def get_info(self):
        print('NAME')
        print(self.name, '\n')
        print('QUANTITY')
        print(self.quantity, '\n')
        print('Market Environment')
        print('\n**Constants**')
        for key, value in self.mar_env.constants.items():
            print(key, value)
        print(']n**Lists**')
        for key, value in self.mar_env.lists.items():
            print(key, value)
        print('\n**Curves')
        for key, value in self.mar_env.constants.items():
            print(key, value)
        print('\nOPTION TYPE')
        print(self.otype, '\n')
        print('PAYOFF FUNCTION')
        print(self.payoff_func)

In [9]:
import datetime as dt
me_gbm = market_environment('me_gbm', dt.datetime(2020, 1, 1))

In [12]:
me_gbm.add_constant('initial_value', 36.)
me_gbm.add_constant('volatility', 0.2)
me_gbm.add_constant('currency', 'EUR')

me)